<a href="https://colab.research.google.com/github/naman015711/capstone-summer-analytics/blob/main/capstone_project_summer_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load your data
df = pd.read_csv('/dataset.csv')  # replace with your actual filename

# Combine date and time correctly
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Clean column names
df.columns = df.columns.str.strip().str.replace(' ', '')

# Vehicle weight mapping
vehicle_weight_map = {'bike': 0.5, 'car': 1.0, 'truck': 1.5, 'cycle': 0.3}
df['vehicle_weight'] = df['VehicleType'].map(vehicle_weight_map)

# Traffic mapping
traffic_map = {'low': 0.5, 'average': 1.0, 'high': 1.5}
df['traffic_level'] = df['TrafficConditionNearby'].map(traffic_map)

# Occupancy ratio
df['occupancy_ratio'] = df['Occupancy'] / df['Capacity']


In [ ]:
BASE_PRICE = 10
ALPHA = 5

# Sort by location and timestamp
df = df.sort_values(['SystemCodeNumber', 'timestamp'])

# Function to apply linear pricing model
def apply_model1(group):
    price = BASE_PRICE
    prices = []
    for occ_ratio in group['occupancy_ratio']:
        price = price + ALPHA * occ_ratio
        price = max(5, min(20, price))  # Clamp between $5 and $20
        prices.append(price)
    group['price_model_1'] = prices
    return group

# Apply per parking lot
df = df.groupby('SystemCodeNumber', group_keys=False).apply(apply_model1)


/tmp/ipython-input-8-2508807673.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('SystemCodeNumber', group_keys=False).apply(apply_model1)


In [ ]:
# Coefficients (feel free to tune)
a, b, c, d, e = 2.0, 1.0, 0.5, 3.0, 1.5
LAMBDA = 0.6

# Compute demand score
df['demand_score'] = (
    a * df['occupancy_ratio'] +
    b * df['QueueLength'] -
    c * df['traffic_level'] +
    d * df['IsSpecialDay'] +
    e * df['vehicle_weight']
)

# Normalize demand score (to [0, 1])
min_demand = df['demand_score'].min()
max_demand = df['demand_score'].max()
df['norm_demand'] = (df['demand_score'] - min_demand) / (max_demand - min_demand)

# Compute price
df['price_model_2'] = BASE_PRICE * (1 + LAMBDA * df['norm_demand'])

# Clamp prices to [5, 20]
df['price_model_2'] = df['price_model_2'].clip(lower=5, upper=20)


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
output_notebook()

# Select a lot to visualize
lot_id = df['SystemCodeNumber'].iloc[0]
plot_data = df[df['SystemCodeNumber'] == lot_id].sort_values('timestamp')

# Create the figure
p = figure(x_axis_type="datetime", title=f"Pricing Over Time – Lot {lot_id}", width=800, height=300)

# Add both pricing models
p.line(plot_data['timestamp'], plot_data['price_model_1'], color='blue', legend_label='Model 1', line_width=2)
p.line(plot_data['timestamp'], plot_data['price_model_2'], color='green', legend_label='Model 2', line_width=2)

# Add hover tool
p.add_tools(HoverTool(
    tooltips=[("Time", "@x{%F %T}"), ("Price", "@y")],
    formatters={'@x': 'datetime'},
    mode='vline'
))

# Axis labels and legend
p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"

# Show the plot
show(p)
